# Setup enviorment

In [1]:
# Data reading in Dataframe format and data preprocessing
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Linear algebra operations
import numpy as np

# Image processing
from skimage import io
from skimage.transform import resize

# Machine learning models and preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional, Flatten
import tensorflow.keras.backend as K

# Epiweek
from epiweeks import Week, Year

# Date
from datetime import date as convert_to_date

# Os
import os

# Feature Extraction Model:
from Variational_Autoencoder_Architecture import get_Variational_Autoencoder

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
features = 'DATASET_5_best_cities/'
labels = 'Tabular_data/dengue_tabular.csv'
Municipalities = ['Medellín', 'Cali', 'Villavicencio', 'Cúcuta', 'Ibagué']

target_size = (224, 224, 12)
backbone = 'Models/vae_224_1024.h5'

cities =  {
  "76001": "Cali",
  "05001": "Medellín",
  "50001": "Villavicencio",
  "54001": "Cúcuta",
  "73001": "Ibagué",
  "68001": "Bucaramanga",
  "05360": "Itagüí",
  "08001": "Barranquilla",
  "41001": "Neiva",
  "23001": "Montería"
}

codes =  {
  "Cali": "76001",
  "Medellín": "05001",
  "Villavicencio": "50001",
  "Cúcuta": "54001",
  "Ibagué": "73001",
  "Bucaramanga": "68001",
  "Itagüí": "05360",
  "Barranquilla": "08001",
  "Neiva": "41001",
  "Montería": "23001"
}

# Read Data

In [4]:
def epiweek_from_date(image_date):
    date = image_date.split('-')
    
    # Get year as int
    year = ''.join(filter(str.isdigit, date[0]))
    year = int(year)
    
    # Get month as int
    month = ''.join(filter(str.isdigit, date[1]))
    month = int(month)
    
    # Get day as int
    day = ''.join(filter(str.isdigit, date[2]))
    day = int(day)
    
    # Get epiweek:
    date = convert_to_date(year, month, day)
    epiweek = str(Week.fromdate(date))
    epiweek = int(epiweek)
    
    return epiweek

In [5]:
def get_epiweek(name):
    
    # Get week
    week = name.split('/')[1]
    week = week.replace('w','')
    week = int(week)
    
    # Year
    year = name.split('/')[0]
    year = int(year)
    
    epiweek = Week(year, week)
    
    epiweek = str(epiweek)
    epiweek = int(epiweek)

    return epiweek

In [6]:
def read_labels(path, Municipality = None):
    df = pd.read_csv(path)
    if df.shape[1] > 678:
        df = pd.concat([df[['Municipality code', 'Municipality']], df.iloc[:,-676:]], axis=1)
        cols = df.iloc[:, 2:].columns
        new_cols = df.iloc[:, 2:].columns.to_series().apply(get_epiweek)
        df = df.rename(columns=dict(zip(cols, new_cols))) 
        
    if 'Label_CSV_All_Municipality' in path:
        # Get Columns
        df = df[['epiweek', 'Municipality code', 'Municipality', 'final_cases_label']]
        
        # change epiweek format
        df.epiweek = df.epiweek.apply(get_epiweek)
        
        # Remove duplicates
        df = df[df.duplicated(['epiweek','Municipality code','Municipality']) == False]
        
        # Replace Increase, decrease, stable to numerical:
        """
        - Stable = 0
        - Increased = 1 
        - Decreased = 2
        """
        df.final_cases_label = df.final_cases_label.replace({'Stable': 0, 'Increased': 1, 'Decreased': 2})
        
        # Create table
        df = df.pivot(index=['Municipality code', 'Municipality'], columns='epiweek', values='final_cases_label')

        # Reset Index:
        df = df.reset_index()
    
    if Municipality:
        df = df[df['Municipality'] == Municipality]
        df.drop(columns=['Municipality code'], inplace=True)
        df.rename(columns={'Municipality': 'Municipality Code'}, inplace=True)
    
        df = df.set_index('Municipality Code')
        df = df.T

        df.columns.name = None
        df.index.name = None
        
        df.columns = ['Cases']
    
    #df = df.reset_index()
    #df.rename(columns={'index': 'epiweek'}, inplace=True)
    return df

### Labels

In [7]:
labels_df = [read_labels(path=labels, Municipality=municipality) for municipality in Municipalities]
labels_df

[        Cases
 200701      1
 200702      0
 200703      0
 200704      0
 200705      0
 ...       ...
 201948     15
 201949     20
 201950     30
 201951     14
 201952      5
 
 [676 rows x 1 columns],
         Cases
 200701      7
 200702      0
 200703      3
 200704      4
 200705      0
 ...       ...
 201948    186
 201949    212
 201950    223
 201951    242
 201952    109
 
 [676 rows x 1 columns],
         Cases
 200701     10
 200702      7
 200703     21
 200704     19
 200705     16
 ...       ...
 201948    103
 201949     99
 201950     66
 201951     34
 201952     25
 
 [676 rows x 1 columns],
         Cases
 200701     82
 200702     63
 200703     63
 200704     70
 200705    100
 ...       ...
 201948     51
 201949     43
 201950     46
 201951     35
 201952     10
 
 [676 rows x 1 columns],
         Cases
 200701     16
 200702     15
 200703     13
 200704     12
 200705     17
 ...       ...
 201948    149
 201949    171
 201950    175
 201951    116
 201952

### Features

In [8]:
def create_df(images_dir):
    
    out_df = {
        'epiweek':[],
        'image':[]
    }
    
    for image_path in os.listdir(images_dir):
        if image_path.endswith('.tiff'):
            epiweek = epiweek_from_date(image_path)
            full_path = os.path.join(images_dir, image_path)
            
            out_df['epiweek'].append(epiweek)
            out_df['image'].append(full_path)

    df = pd.DataFrame(out_df)
    
    df = df.set_index('epiweek')
    df.index.name = None
    
    return df

In [9]:
if 'DATASET_5_best_cities' not in features:
    for i in range(len(Municipalities)):
        Municipalities[i] = codes[Municipalities[i]]
    
images_dir_list = [os.path.join(features, MUNICIPALITY) for MUNICIPALITY in Municipalities]

features_df = [create_df(images_dir) for images_dir in images_dir_list]
features_df[0].head()

,image
201731,DATASET_5_best_cities/Medellín/image_2017-07-3...
201551,DATASET_5_best_cities/Medellín/image_2015-12-2...
201747,DATASET_5_best_cities/Medellín/image_2017-11-1...
201647,DATASET_5_best_cities/Medellín/image_2016-11-2...
201729,DATASET_5_best_cities/Medellín/image_2017-07-1...


In [10]:
def read_image(path, target_size):
    # Read the image and convert to numpy array
    image = io.imread(path)
    # Resize the image and normalize values
    image_arr = resize(image,(target_size[0], target_size[1]))
    #print(f'The shape of the image before reshape: {image_arr.shape}, of type{type(image_arr)}')

    # Select RGB bands
    if target_size[2] == 3:
        image_arr = image_arr[:,:, [1,2,3]]
    return image_arr

In [11]:
for i in range(len(features_df)):
    features_df[i].image = features_df[i].image.apply(read_image, args=[target_size])
#features_df.head()

# Data preparation

In [12]:
n_labels = labels_df[0].shape[1]

In [13]:
# Merge the two dataframes based on the date values
dengue_df = [features_df[i].merge(labels_df[i], how='inner', left_index=True, right_index=True) for i in range(len(labels_df))]
dengue_df = [dengue_df[i].sort_index() for i in range(len(dengue_df))]
#dengue_df.head()

### Train Test split

In [14]:
def train_test_split(df, train_percentage = 80):
    # We need a sequence so we can't split randomly
    # To divide into Train and test we have to calculate the train percentage of the dataset:
    size = df.shape[0]
    split = int(size*(train_percentage/100))
    
    """ Train """
    # We will train with 1st percentage % of data and test with the rest
    train_df = df.iloc[:split,:] ## percentage % train
    
    """ Test """
    test_df = df.iloc[split:,:] # 100 - percentage % test
    
    print(f'The train shape is: {train_df.shape}')
    print(f'The test shape is: {test_df.shape}')
    
    return train_df, test_df

In [15]:
train_df = []
test_df = []

for i in range(len(dengue_df)):
    train_df_aux, test_df_aux = train_test_split(dengue_df[i], train_percentage = 80)
    train_df.append(train_df_aux)
    test_df.append(test_df_aux)
#test_df

The train shape is: (132, 2)
The test shape is: (33, 2)
The train shape is: (132, 2)
The test shape is: (33, 2)
The train shape is: (132, 2)
The test shape is: (33, 2)
The train shape is: (132, 2)
The test shape is: (33, 2)
The train shape is: (132, 2)
The test shape is: (33, 2)


### Normalization

In [16]:
def normalize_train_labels(df, column, feature_range=(-1, 1)):
    # Get values of the column
    values = df[column].values.reshape(-1,1)
    # Generate a new scaler
    scaler = MinMaxScaler(feature_range=feature_range)
    # Fit the scaler just for that column
    scaled_column = scaler.fit_transform(values)
    # Add the scaled column to the dataframe
    scaled_column = np.reshape(scaled_column, len(scaled_column))
    df[column] = scaled_column
    return df, scaler
    
def normalize_test_labels(df, column, scaler):
    # Get values of the column
    values = df[column].values.reshape(-1,1)
    # Scale values
    scaled_column = scaler.transform(values)
    scaled_column = np.reshape(scaled_column,len(scaled_column))
    # Add the scaled values to the df
    df[column] = scaled_column
    return df
    

In [17]:
# Merge:
train_df = pd.concat([train_df[0], train_df[1], train_df[2], train_df[3], train_df[4]], keys=Municipalities)
test_df = pd.concat([test_df[0], test_df[1], test_df[2], test_df[3], test_df[4]], keys=Municipalities)

In [18]:
feature_range = (-1, 1)

# Scale train:
train_df, scaler = normalize_train_labels(train_df, 'Cases', feature_range=feature_range)
train_df = [train_df[train_df.index.get_level_values(0) == municipality] for municipality in Municipalities]

#train_df[0].head()

In [19]:
feature_range = (-1, 1)

# Scale test:
test_df = normalize_test_labels(test_df, 'Cases', scaler=scaler)
test_df = [test_df[test_df.index.get_level_values(0) == municipality] for municipality in Municipalities]

#test_df[0].head()

### Prepare data for time series supervised learning (function to create sliding window)

In [20]:
# prepare data for time series

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True, no_autoregressive=None):
    if no_autoregressive:
        n_in = n_in - 1
        
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        if no_autoregressive:
            cols.append(df.shift(i).iloc[:,:-1])
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars-1)]
        else:
            cols.append(df.shift(i))
            names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [21]:
# length of window
days = 10
no_autoregressive = True

# frame as supervised learning
train = [series_to_supervised(df, n_in=days, no_autoregressive=no_autoregressive) for df in train_df]
test = [series_to_supervised(df, n_in=days, no_autoregressive=no_autoregressive) for df in test_df]

#DataFrame(train[0])

### Merge train data

In [22]:
# Merge:
train = pd.concat([train[0], train[1], train[2], train[3], train[4]], keys=Municipalities)
test = pd.concat([test[0], test[1], test[2], test[3], test[4]], keys=Municipalities)

In [23]:
def convert_df_to_np(train):
    for i, column in enumerate(train.columns):
        if i == 0:
            train_arr = np.array(train[column].to_list())
            train_arr = np.expand_dims(train_arr, axis=1)

        else:
            #print(f'original: {train_arr.shape}')

            train_arr_aux = np.array(train[column].to_list())
            train_arr_aux = np.expand_dims(train_arr_aux, axis=1)

            #print(f'aux: {train_arr_aux.shape}')

            train_arr = np.concatenate((train_arr, train_arr_aux), axis=1)

    return train_arr

### Features and Labels Set

In [24]:
def features_labels_set(timeseries_data, original_df):
    
    """ Features """
    # We define the number of features as (Cases and media cloud)
    n_features = original_df.shape[1]

    # The features to train the model will be all except the values of the actual week 
    # We can't use other variables in week t because whe need to resample a a 3D Array
    features_set = DataFrame(timeseries_data.values[:,:-1])
    # Convert pandas data frame to np.array to reshape as 3D Array
    features_set = convert_df_to_np(features_set)
    print(f'The shape of the features is {features_set.shape}')
    
    """ Labels """
    # We will use Covid cases in last week 
    labels_set = DataFrame(timeseries_data.values[:,-1])
    # Convert pandas data frame to np.array
    labels_set = labels_set.to_numpy()
    print(f'The shape of the labels is {labels_set.shape}')
    
    return features_set, labels_set, n_features

In [25]:
# Train features and labels set
print('Train:')
train_X, train_y, n_features = features_labels_set(timeseries_data=train, original_df=dengue_df[0])

# Test features and labels set
print('Test:')
test_X, test_y, n_features = features_labels_set(timeseries_data=test, original_df=dengue_df[0])

Train:
The shape of the features is (615, 10, 224, 224, 12)
The shape of the labels is (615, 1)
Test:
The shape of the features is (120, 10, 224, 224, 12)
The shape of the labels is (120, 1)


In [26]:
train_X = np.asarray(train_X).astype(np.float32)
train_y = np.asarray(train_y).astype(np.float32)

test_X = np.asarray(test_X).astype(np.float32)
test_y = np.asarray(test_y).astype(np.float32)

# Define the Model

In [27]:
# Set Seed
#tf.random.set_seed(0)

def smape(y_true, y_pred):
    epsilon = 0.1
    summ = K.maximum(K.abs(y_true) + K.abs(y_pred) + epsilon, 0.5 + epsilon)
    smape = K.abs(y_pred - y_true) / summ * 2.0
    return smape


def create_model(backbone=backbone):
    lstm_week, input_shape = days, target_size
    
    # design network
    model = Sequential()

    # CNN
    cnn = get_Variational_Autoencoder(model_path=backbone, backbone=True)

    for idx, layer in enumerate(cnn.layers):
        layer.trainable = False # idx > len(cnn.layers) - 2 
    
    # https://levelup.gitconnected.com/hands-on-practice-with-time-distributed-layers-using-tensorflow-c776a5d78e7e
    model.add(keras.layers.TimeDistributed(cnn, input_shape = ((lstm_week,) + input_shape)))
    model.add(keras.layers.TimeDistributed(Flatten()))
    model.add(keras.layers.TimeDistributed(Dense(1024)))
    model.add(LSTM(120, dropout=0.1, return_sequences=True))
    model.add(LSTM(240, dropout=0.1, return_sequences = False))
    model.add(Dense(60))
    model.add(Dense(1))
    
    # Compile the model:
    opt = keras.optimizers.Adam()
    
    # Metrics
    metrics = [
        tf.keras.metrics.RootMeanSquaredError(name='rmse'),
        tf.keras.metrics.MeanAbsolutePercentageError(name='mape'),
        smape
    ]
    
    model.compile(loss='mse', optimizer=opt, metrics=metrics)

    return model

### Train the model

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

# EarlyStopping:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=20, 
        verbose=1, mode='auto', restore_best_weights=True)

In [29]:
# fit network
def train_model(model, monitor, plot=None, epochs=20):
    if monitor:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False, callbacks=[monitor])
    else:
        history = model.fit(train_X, train_y, epochs=epochs, batch_size=16, validation_data=(test_X, test_y), verbose=2, shuffle=False)
    
    if plot:
        # plot history
        plt.plot(history.history['loss'], label='train')
        plt.plot(history.history['val_loss'], label='validation')
        plt.legend()
        plt.show()

# Test the model

In [30]:
from math import sqrt
from numpy import concatenate

def test_model(model, test_X, test_y, scaler, rnn = None):
    
    # If model is a classical machine learning model and test_X is a 3D tensor, then convert to 2D
    if not rnn and (len(test_X.shape) == 3):
        test_X = test_X.reshape((test_X.shape[0], -1))
    
    # do the prediction
    yhat = model.predict(test_X)
    
    # Invert scaling for forecast
    # Inverse Scaler
    
    # Predicted
    if not rnn:
        yhat = yhat.reshape(-1, 1)
        
    if not scaler:
        return yhat, test_y
    
    inv_yhat = scaler.inverse_transform(yhat)
    
    # Real:
    inv_y = scaler.inverse_transform(test_y)
    
    return inv_yhat, inv_y

### Mean Absolute Percentage Error (MAPE)

$$
MAPE = \displaystyle\frac{100\%}{n}\sum_{t=1}^{n}\left |\frac{x_i-y_i}{y_t}\right|
$$

MAPE has a problem if there are zeros in the test data, so other metrics can be explored

In [31]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Test MAPE: %.3f' % mape)
    return mape

### Symmetric Mean Absolute Percentage Error (sMAPE)

$$
sMAPE = \displaystyle\frac{100\%}{n}\sum_{t=1}^{n} \frac{|x_i-y_i|}{|x_i|+|y_t|}
$$

In [32]:
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    smape = 1/len(y_true) * np.sum(2 * np.abs(y_pred-y_true) / (np.abs(y_true) + np.abs(y_pred))*100)
    print('Test sMAPE: %.3f' % smape)
    return smape

### Root Mean Squared Error (RMSE)
$$
RMSE = \sqrt{(\frac{1}{n})\sum_{i=1}^{n}(x_i-y_i)^{2}}
$$

In [33]:
from sklearn.metrics import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    print('Test RMSE: %.3f' % rmse)
    return rmse

In [34]:
def plot_predictions(inv_y, inv_yhat, model_name = ''):
    data_predict = inv_yhat  ## predicted target cases
    dataY_plot = inv_y  ##  real test-target cases

    data_predict = data_predict.reshape(len(data_predict), 1)
    dataY_plot = dataY_plot.reshape(len(dataY_plot), 1)

    plt.plot(dataY_plot, label = 'actual')
    plt.plot(data_predict, label = 'predicted')
    plt.legend(loc="upper left")
    plt.suptitle(f'Time-Series Prediction with {model_name}')
    plt.show()

In [35]:
def evaluate(model, test_X, test_y, scaler):
    stored_results = {}
    inv_yhat_lstm, inv_y_lstm = test_model(model=model, test_X=test_X, test_y=test_y, scaler=scaler, rnn = True)
    stored_results['mape'] = mean_absolute_percentage_error(inv_y_lstm, inv_yhat_lstm)
    stored_results['smape'] = symmetric_mean_absolute_percentage_error(inv_y_lstm, inv_yhat_lstm)
    stored_results['rmse'] = root_mean_squared_error(inv_y_lstm, inv_yhat_lstm)

    return stored_results

In [36]:
"""
model = create_model(backbone=backbone)
train_model(model=model, monitor=monitor)

model.save(f'Models/{backbone}_LSTM_Regression.h5')
model.summary()

inv_yhat_lstm, inv_y_lstm = test_model(model=model, test_X=test_X, test_y=test_y, scaler=scaler, rnn = True)

evaluate(model, test_X, test_y, scaler)

# LSTM
plot_predictions(inv_y_lstm, inv_yhat_lstm, model_name = 'LSTM')
"""

"\nmodel = create_model(backbone=backbone)\ntrain_model(model=model, monitor=monitor)\n\nmodel.save(f'Models/{backbone}_LSTM_Regression.h5')\nmodel.summary()\n\ninv_yhat_lstm, inv_y_lstm = test_model(model=model, test_X=test_X, test_y=test_y, scaler=scaler, rnn = True)\n\nevaluate(model, test_X, test_y, scaler)\n\n# LSTM\nplot_predictions(inv_y_lstm, inv_yhat_lstm, model_name = 'LSTM')\n"

# Calculate Mean and SD

In [37]:
# With LSTM:
#print(f'The scalers are: {scalers.keys()}')
#y_scaler = scalers['scaler_Cases']

def calculate_mean_std():
    
    metrics = {
        "rmse": [],
        "mape": [],
        "smape": []
    }
    
    for i in range(5):
        model = create_model(backbone=backbone)
        train_model(model=model, monitor=monitor)
        stored_results = evaluate(model, test_X, test_y, scaler)
        print(stored_results)
        
        for key in metrics.keys():
            metrics[key].append(stored_results[key])
            
    for key in metrics.keys():
        results = metrics[key]
        print(key, f": average={np.average(results):.3f}, std={np.std(results):.3f}")


In [ ]:
calculate_mean_std()

Instructions for updating:
Colocations handled automatically by placer.
Train on 615 samples, validate on 120 samples
Epoch 1/20
615/615 - 18s - loss: 0.2555 - rmse: 0.5055 - mape: 81.3880 - smape: 0.5754 - val_loss: 0.0450 - val_rmse: 0.2121 - val_mape: 19.2587 - val_smape: 0.1713
Epoch 2/20
615/615 - 17s - loss: 0.1586 - rmse: 0.3982 - mape: 92.4030 - smape: 0.4550 - val_loss: 0.0204 - val_rmse: 0.1430 - val_mape: 12.8308 - val_smape: 0.1184
Epoch 3/20
615/615 - 17s - loss: 0.1618 - rmse: 0.4023 - mape: 102.4439 - smape: 0.4240 - val_loss: 0.0195 - val_rmse: 0.1397 - val_mape: 12.6220 - val_smape: 0.1227
Epoch 4/20
615/615 - 17s - loss: 0.1478 - rmse: 0.3845 - mape: 94.7670 - smape: 0.4048 - val_loss: 0.0219 - val_rmse: 0.1480 - val_mape: 13.9177 - val_smape: 0.1395
Epoch 5/20
615/615 - 17s - loss: 0.1394 - rmse: 0.3733 - mape: 91.9032 - smape: 0.3862 - val_loss: 0.0169 - val_rmse: 0.1300 - val_mape: 12.3871 - val_smape: 0.1187
Epoch 6/20
615/615 - 17s - loss: 0.1453 - rmse: 0.3812 -